In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import pandas_udf
from pyspark.sql import SparkSession

In [2]:
def plot_subjects(df,colName, title, xlabel):   
    fontdict = {"fontsize":20}
    plt.barh(df.index, df[colName],)
    plt.title(title,fontdict)
    plt.ylabel('Subject ID',fontdict)
    plt.xlabel(xlabel,fontdict)
    plt.show()
    
def plot_boxplot(df, title, ylabel, xlabel):
    fontdict = {"fontsize":20}
    plt.title(title)
    plt.title(title,fontdict)
    plt.ylabel(ylabel,fontdict)
    plt.xlabel(xlabel,fontdict)
    sns.boxplot(data=df, width = 0.5,orient="horizontal") 

In [2]:
cleaned_data = pd.read_csv("data/cleaned_data.csv")

In [117]:
cleaned_data

,Bed,Bath,m²,Price,Address,Property_type,New_build,Page,County,Province
0,2.0,1.0,52.0,295000,"8 Parknasilla, Vevay Road, Bray, Co. Wicklow",Apartment,no,0,Wicklow,Leinster
1,3.0,3.0,93.0,230000,"7 Meadow Avenue, The Meadows, Hollyhill, Co. Cork",Semi-D,no,0,Cork,Munster
2,4.0,3.0,86.0,265000,"5 Friar Street, Cork City, Co. Cork",Terrace,no,0,Cork,Munster
3,3.0,1.0,82.0,375000,"12 Barrack Street, Kinsale, Co. Cork",End of Terrace,no,0,Cork,Munster
4,5.0,4.0,186.0,405000,"3 The Garden, Whitefield Manor, Bettystown, Co...",Semi-D,no,0,Meath,Leinster
...,...,...,...,...,...,...,...,...,...,...
19759,4.0,3.0,128.0,PoA,"The Oak - 4 Bed Semi-Detached, Cluain Dara, Cl...",NaN,no,983,Kildare,Leinster
19760,3.0,3.0,105.0,365000,"House Type D, Castle Rock, Castle Rock, Castle...",Semi-D,no,983,Limerick,Munster
19761,1.0,1.0,58.0,260000,"Elder / Ash, Hillfort, Hillfort , Kill, Co. K...",NaN,no,983,Kildare,Leinster
19762,3.0,3.0,103.0,425000,"Elm, Hillfort, Hillfort , Kill, Co. Kildare",NaN,no,983,Kildare,Leinster


In [4]:
cleaned_data.pivot_table(columns="Property_type")

C:\Users\PC\AppData\Local\Temp\ipykernel_7748\3417761224.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  cleaned_data.pivot_table(columns="Property_type")


Property_type,Apartment,Bungalow,Detached,Duplex,End of Terrace,House,Semi-D,Site,Studio,Terrace,Townhouse
Bath,1.662768,1.843489,2.686930,2.334842,2.033696,1.0,2.290339,NaN,1.000000,1.910900,2.089796
Bed,2.058880,3.375144,3.988089,2.977578,3.140389,3.0,3.389085,NaN,NaN,3.027280,3.096774
Page,395.366265,469.272727,496.074667,412.125561,406.444564,703.0,386.588147,761.079479,327.333333,409.075680,497.048387
m²,76.177270,668.662791,31388.957548,111.293478,116.215377,NaN,156.171675,102365.925594,34.000000,109.433252,165.228430


In [5]:
cleaned_data.groupby("Property_type").mean()

C:\Users\PC\AppData\Local\Temp\ipykernel_7748\310290819.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  cleaned_data.groupby("Property_type").mean()


,Bed,Bath,m²,Page
Property_type,,,,
Apartment,2.058880,1.662768,76.177270,395.366265
Bungalow,3.375144,1.843489,668.662791,469.272727
Detached,3.988089,2.686930,31388.957548,496.074667
Duplex,2.977578,2.334842,111.293478,412.125561
End of Terrace,3.140389,2.033696,116.215377,406.444564
House,3.000000,1.000000,NaN,703.000000
Semi-D,3.389085,2.290339,156.171675,386.588147
Site,NaN,NaN,102365.925594,761.079479
Studio,NaN,1.000000,34.000000,327.333333


In [210]:
def fix_price(price):
    try:
        return int(price)
    except ValueError:
        return np.nan
        
def fix_floor_area(area):
    ft2m = 10.764
    median_area = 110
    try:
        if len(area.split()) == 2:
            if area.split()[0][-1] == "m":
                return area.split()[0][:-1]
            elif area.split()[0][-2:] == "ft":
                return round((float(area[:-4])/ft2m),2)
            count += 1
        else:
            area1 = float(area.split(" - ")[0])
            area2 = float(area.split(" - ")[1][:-3])
            
            if np.abs(median_area - area1) <= np.abs(median_area - area2):
                return area1
            else:
                return area2
    except AttributeError:
        pass
    
def find_county(address: str, counties: np.ndarray) -> str:

    """
    The address is comma seperated with the county at the end. (Some adresses are missing the county).
    We use the array counties to check if the county found is actually a county.

    We also need to check for counties such as Dublin 1, Dublin 2 etc.

    """
    countyName = address.split(",")[-1].split()
    if len(countyName) == 1:
        if countyName[0].strip().capitalize() in counties or countyName[0].strip().capitalize()[:-1] in counties:
            return countyName[0].capitalize()
        
        elif len(countyName[0].strip().lower().split("co.")) > 1:
            return countyName[0][3:].capitalize()
        
        else:
            return np.nan
        
    elif len(countyName) == 2:
        if countyName[0].strip().capitalize() in counties or countyName[0].strip().capitalize()[:-1] in counties:
            try:
                int(countyName[1].strip())
                return " ".join(countyName)
            
            except ValueError:
                return countyName[0].strip().capitalize()
            
        elif countyName[1].strip().capitalize() in counties or countyName[1].strip().capitalize()[:-1] in counties:
            return countyName[1].strip().capitalize()
        
        else:
            return np.nan
        
    else:
        for value in countyName:
            if value.strip().capitalize() in counties or value.strip().capitalize()[:-1] in counties:
                return value
            
def fix_counties(county):
    try:
        if county[-1] == "." or county[-1] == "-":
            return county[:-1]
        else:
            return county
    except TypeError:
        return np.nan 
    
def find_province(county: str, county_dict: dict) -> str:

    """"
    This function aims to find the province using the county found with the last function.

    It takes the county in and dictionary to return the correct province.
    """

    if county in county_dict: # Works for Counties outside of Dublin
        return county_dict[county]
    elif county.split(" ")[0] == "Dublin": # Works for Dublin
        return county_dict["Dublin"]
    else:
        print(f"County: {county} was not found in dictionary")

def create_county_dict(df: pd.DataFrame) -> dict:

    """
    Takes in dataframe with the columns County and Province,
    then returns a dictionary with the county as the key and the province as the value.
    This dictionary is used in the function above.
    """

    county_dict = {}
    for i in range(len(df)):
        county_dict[df["County"][i]] = df["Province"][i]
    return county_dict

In [350]:
daft = pd.read_csv("data/daft_from_page_0_till_page_987_by_20.csv", index_col=["Unnamed: 0"])
daft

,Bed,Bath,m²,Price,Address,Property_type,New_build,Page
0,2.0,1.0,52.0,295000,"8 Parknasilla, Vevay Road, Bray, Co. Wicklow",Apartment,no,0
1,3.0,3.0,93.0,230000,"7 Meadow Avenue, The Meadows, Hollyhill, Co. Cork",Semi-D,no,0
2,4.0,3.0,86.0,265000,"5 Friar Street, Cork City, Co. Cork",Terrace,no,0
3,3.0,1.0,82.0,375000,"12 Barrack Street, Kinsale, Co. Cork",End of Terrace,no,0
4,5.0,4.0,186.0,405000,"3 The Garden, Whitefield Manor, Bettystown, Co...",Semi-D,no,0
...,...,...,...,...,...,...,...,...
19840,4.0,3.0,128.0,Price on Application,"The Oak - 4 Bed Semi-Detached, Cluain Dara, Cl...",NaN,no,983
19841,3.0,3.0,105.0,365000,"House Type D, Castle Rock, Castle Rock, Castle...",Semi-D,no,983
19842,1.0,1.0,58.0,260000,"Elder / Ash, Hillfort, Hillfort , Kill, Co. K...",NaN,no,983
19843,3.0,3.0,103.0,425000,"Elm, Hillfort, Hillfort , Kill, Co. Kildare",NaN,no,983


In [339]:
province_df = pd.read_csv("data/county_data.csv")
county_dict = create_county_dict(province_df)

In [348]:
daft["Price"] = daft["Price"].apply(lambda x :fix_price(x))
daft["County"] = daft["Address"].apply(lambda x :find_county(x, county_dict))
daft.dropna(subset="County", inplace=True)
daft["Province"] = daft["County"].apply(lambda x: find_province(x, county_dict))

In [349]:
daft

,Bed,Bath,m²,Price,Address,Property_type,New_build,Page,County,Province
0,2.0,1.0,52.0,295000.0,"8 Parknasilla, Vevay Road, Bray, Co. Wicklow",Apartment,no,0,Wicklow,Leinster
1,3.0,3.0,93.0,230000.0,"7 Meadow Avenue, The Meadows, Hollyhill, Co. Cork",Semi-D,no,0,Cork,Munster
2,4.0,3.0,86.0,265000.0,"5 Friar Street, Cork City, Co. Cork",Terrace,no,0,Cork,Munster
3,3.0,1.0,82.0,375000.0,"12 Barrack Street, Kinsale, Co. Cork",End of Terrace,no,0,Cork,Munster
4,5.0,4.0,186.0,405000.0,"3 The Garden, Whitefield Manor, Bettystown, Co...",Semi-D,no,0,Meath,Leinster
...,...,...,...,...,...,...,...,...,...,...
19840,4.0,3.0,128.0,NaN,"The Oak - 4 Bed Semi-Detached, Cluain Dara, Cl...",NaN,no,983,Kildare,Leinster
19841,3.0,3.0,105.0,365000.0,"House Type D, Castle Rock, Castle Rock, Castle...",Semi-D,no,983,Limerick,Munster
19842,1.0,1.0,58.0,260000.0,"Elder / Ash, Hillfort, Hillfort , Kill, Co. K...",NaN,no,983,Kildare,Leinster
19843,3.0,3.0,103.0,425000.0,"Elm, Hillfort, Hillfort , Kill, Co. Kildare",NaN,no,983,Kildare,Leinster


In [ ]:

myhome_listings["County"] = myhome_listings["County"].apply(lambda x: fix_counties(x))

myhome_listings.dropna(subset=["County"], inplace=True) # Haven't tested this so it maybe wrong.


In [338]:
daft.describe()

,Bed,Bath,m²,Page
count,16705.000000,16435.000000,1.545800e+04,19845.000000
mean,3.362766,2.259690,2.813382e+04,488.017133
std,1.430034,1.345817,2.372859e+06,285.353922
min,1.000000,1.000000,0.000000e+00,0.000000
25%,3.000000,1.000000,9.000000e+01,241.000000
50%,3.000000,2.000000,1.280000e+02,488.000000
75%,4.000000,3.000000,2.680000e+02,735.000000
max,40.000000,32.000000,2.630408e+08,983.000000


In [224]:
myhome_listings

,beds,baths,floor_area,Price,Address,Property_type,New_build,Page
0,3.0,3.0,NaN,PoA,"3 Bedroom Apartment - 55 Lad Lane, Dublin 2",Apartment,NaN,1.0
1,3.0,1.0,105m 2,550000,"38 The Avenue, Boden Park, Rathfarnham, Dubli...",Semi-Detached House,NaN,1.0
2,4.0,3.0,110m 2,795000,"130 Carysfort Park, Blackrock, County Dublin",Semi-Detached House,NaN,1.0
3,2.0,1.0,68m 2,295000,"5 Woodbrook Hall, Carpenterstown, Dublin 15",Apartment,NaN,1.0
4,4.0,2.0,170.07m 2,895000,"70 Kincora Grove, Clontarf, Dublin 3",Semi-Detached House,NaN,1.0
...,...,...,...,...,...,...,...,...
9995,5.0,5.0,200m 2,365000,"No. 1 Gleann Na Boirne, Bellharbour, Clare",Detached House,NaN,500.0
9996,3.0,1.0,107m 2,419000,"Knockdilly Cottage, Knockdilly, Kilmuckridge,...",Detached House,NaN,500.0
9997,4.0,3.0,2675ft 2,435000,"Dooneen, Carrigtwohill, Cork",Detached House,NaN,500.0
9998,2.0,3.0,75m 2,197500,"14 Blossom Court, Esker Hills, Portlaoise, Laois",Detached House,NaN,500.0


In [226]:
myhome_listings

,beds,baths,floor_area,Price,Address,Property_type,New_build,Page,County,Province
0,3.0,3.0,None,NaN,"3 Bedroom Apartment - 55 Lad Lane, Dublin 2",Apartment,NaN,1.0,Dublin 2,Leinster
1,3.0,1.0,105,550000.0,"38 The Avenue, Boden Park, Rathfarnham, Dubli...",Semi-Detached House,NaN,1.0,Dublin 14,Leinster
2,4.0,3.0,110,795000.0,"130 Carysfort Park, Blackrock, County Dublin",Semi-Detached House,NaN,1.0,Dublin,Leinster
3,2.0,1.0,68,295000.0,"5 Woodbrook Hall, Carpenterstown, Dublin 15",Apartment,NaN,1.0,Dublin 15,Leinster
4,4.0,2.0,170.07,895000.0,"70 Kincora Grove, Clontarf, Dublin 3",Semi-Detached House,NaN,1.0,Dublin 3,Leinster
...,...,...,...,...,...,...,...,...,...,...
9995,5.0,5.0,200,365000.0,"No. 1 Gleann Na Boirne, Bellharbour, Clare",Detached House,NaN,500.0,Clare,Munster
9996,3.0,1.0,107,419000.0,"Knockdilly Cottage, Knockdilly, Kilmuckridge,...",Detached House,NaN,500.0,Wexford,Leinster
9997,4.0,3.0,248.51,435000.0,"Dooneen, Carrigtwohill, Cork",Detached House,NaN,500.0,Cork,Munster
9998,2.0,3.0,75,197500.0,"14 Blossom Court, Esker Hills, Portlaoise, Laois",Detached House,NaN,500.0,Laois,Leinster


In [211]:
cleaned_data

,Bed,Bath,m²,Price,Address,Property_type,New_build,Page,County,Province
0,2.0,1.0,52.0,295000,"8 Parknasilla, Vevay Road, Bray, Co. Wicklow",Apartment,no,0,Wicklow,Leinster
1,3.0,3.0,93.0,230000,"7 Meadow Avenue, The Meadows, Hollyhill, Co. Cork",Semi-D,no,0,Cork,Munster
2,4.0,3.0,86.0,265000,"5 Friar Street, Cork City, Co. Cork",Terrace,no,0,Cork,Munster
3,3.0,1.0,82.0,375000,"12 Barrack Street, Kinsale, Co. Cork",End of Terrace,no,0,Cork,Munster
4,5.0,4.0,186.0,405000,"3 The Garden, Whitefield Manor, Bettystown, Co...",Semi-D,no,0,Meath,Leinster
...,...,...,...,...,...,...,...,...,...,...
19759,4.0,3.0,128.0,PoA,"The Oak - 4 Bed Semi-Detached, Cluain Dara, Cl...",NaN,no,983,Kildare,Leinster
19760,3.0,3.0,105.0,365000,"House Type D, Castle Rock, Castle Rock, Castle...",Semi-D,no,983,Limerick,Munster
19761,1.0,1.0,58.0,260000,"Elder / Ash, Hillfort, Hillfort , Kill, Co. K...",NaN,no,983,Kildare,Leinster
19762,3.0,3.0,103.0,425000,"Elm, Hillfort, Hillfort , Kill, Co. Kildare",NaN,no,983,Kildare,Leinster


In [261]:

sold_df

,Date,Address,Price,Page
0,2022-11-18,"ELMFIELD, BARNDARRIG, WICKLOW, A67NW63, Co. Wi...",605000,1
1,2022-11-18,"13 TARA COURT, DUBLIN, Balbriggan, Co. Dublin,...",132500,1
2,2022-11-18,"23 Brennanstown Wood, Brennanstown Road, Cabin...",837004,1
3,2022-11-17,"MINDORAN, DONEGAL, F93C3K4, Clonmany, Co. Donegal",96000,1
4,2022-11-17,"3 Chapel Lane, Kildare, Co. Kildare",312775,1
...,...,...,...,...
19995,2022-06-16,"141 Clarion Village Apts, Sligo, Co. Sligo",72000,1000
19996,2022-06-16,"4 Riverbank, Easkey, Co. Sligo, F26et04",195000,1000
19997,2022-06-16,"9 Holycross Cottages, Holycross, Co. Tipperary...",185000,1000
19998,2022-06-16,"2 The Grove, Ardmore, Co. Waterford, P36x093",33333,1000


In [294]:
def find_county(address: str, counties: np.ndarray, county_position=-1) -> str:

    """
    The address is comma seperated with the county at the end. (Some adresses are missing the county).
    We use the array counties to check if the county found is actually a county.

    We also need to check for counties such as Dublin 1, Dublin 2 etc.

    """
    try:
        countyName = address.split(",")[county_position].split()
    except IndexError:
        return np.nan
    if len(countyName) == 1:
        if countyName[0].strip().capitalize() in counties or countyName[0].strip().capitalize()[:-1] in counties:
            return countyName[0].capitalize()
        
        elif len(countyName[0].strip().lower().split("co.")) > 1:
            return countyName[0][3:].capitalize()
        
        else:
            return np.nan
        
    elif len(countyName) == 2:
        if countyName[0].strip().capitalize() in counties or countyName[0].strip().capitalize()[:-1] in counties:
            try:
                int(countyName[1].strip())
                return " ".join(countyName)
            
            except ValueError:
                return countyName[0].strip().capitalize()
            
        elif countyName[1].strip().capitalize() in counties or countyName[1].strip().capitalize()[:-1] in counties:
            return countyName[1].strip().capitalize()
        
        else:
            return np.nan
        
    else:
        for value in countyName:
            if value.strip().capitalize() in counties or value.strip().capitalize()[:-1] in counties:
                return value

In [321]:
sold_df = pd.read_csv("data/myHome_sold_property_from_page_1_till_page_1000.csv", index_col=["Unnamed: 0"])

sold_df_counties_1 = pd.DataFrame(sold_df["Address"].apply(lambda x : find_county(x, county_dict,county_position=-1)).dropna())
sold_df_counties_2 = pd.DataFrame(sold_df["Address"].apply(lambda x : find_county(x, county_dict,county_position=-2)).dropna())

sold_df = sold_df.join(
    sold_df_counties_1.append(
    sold_df_counties_2).sort_index(
    ).rename(columns={"Address":"County"})
    ).drop_duplicates(subset="Address")

sold_df.dropna(subset="County", inplace=True)
sold_df["County"]= sold_df["County"].replace({"Mayor":"Dublin 1"})
sold_df["Province"] = sold_df["County"].apply(lambda x: find_province(x, county_dict))

C:\Users\PC\AppData\Local\Temp\ipykernel_7748\3793288180.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sold_df_counties_1.append(


In [331]:
price_change_df = pd.read_csv("data/myHome_price_change_from_page_1_till_page_349.csv", index_col=["Unnamed: 0"])

price_change_counties_1 = pd.DataFrame(price_change_df["Address"].apply(lambda x : find_county(x, county_dict,county_position=-1)).dropna())
price_change_counties_2 = pd.DataFrame(price_change_df["Address"].apply(lambda x : find_county(x, county_dict,county_position=-2)).dropna())


price_change_df = price_change_df.join(
    price_change_df_counties_1.append(
    price_change_df_counties_2).sort_index(
    ).rename(columns={"Address":"County"})
    ).drop_duplicates(subset="Address")

price_change_df.dropna(subset="County", inplace=True)
price_change_df["County"] = price_change_df["County"].apply(lambda x: fix_counties(x))
price_change_df["County"]= price_change_df["County"].replace({"Downs":"Wicklow"})
price_change_df["Province"] = price_change_df["County"].apply(lambda x: find_province(x, county_dict))

C:\Users\PC\AppData\Local\Temp\ipykernel_7748\2236333316.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  price_change_df_counties_1.append(


In [333]:
price_change_df

,Date,Address,New_price,Old_price,Change(€),Change(%),Page,County,Province
0,2022-11-26,"5 Woodbrook Hall, Carpenterstown, Dublin 15",295000,275000,20000,7.27,1,Dublin 15,Leinster
1,2022-11-25,"18 The Crescent, Cnoc na Greine, Kilcullen, K...",379950,369950,10000,2.70,1,Kildare,Leinster
2,2022-11-25,"9 Hawthorn House, Carrickmines Green, Carrick...",430000,399950,30050,7.51,1,Dublin 18,Leinster
3,2022-11-25,"Ballybricken, Grange, Limerick",99000,100000,-1000,-1.00,1,Limerick,Munster
4,2022-11-25,"5 Woodstown Rise, Knocklyon, Dublin 16",505000,530000,-25000,-4.72,1,Dublin 16,Leinster
...,...,...,...,...,...,...,...,...,...
4584,2021-11-29,"Apt 26, The Schooner, Alverno Clontarf, Co. ...",299000,300000,-1000,-0.33,349,Dublin 3,Leinster
4585,2021-11-27,"31 Cross Avenue, Dun Laoghaire, County Dublin",475000,465000,10000,2.15,349,Dublin,Leinster
4588,2021-11-26,"1 Rollins Villas, Sallynoggin, County Dublin",395000,325000,70000,21.54,349,Dublin,Leinster
4589,2021-11-26,"6 Westbourne View, Clondalkin, Dublin 22",249950,265000,-15050,-5.68,349,Dublin 22,Leinster
